##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [15]:
#%pip install pandas 
#%pip install matplotlib
#%pip install pyarrow
#%pip install fastparquet
#%pip install seaborn
#%pip install scikit-learn

# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [16]:
# Can have as many cells as you want for code
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from datetime import datetime

# Get the current working directory
new_directory = "C:/Users/parma/OneDrive/Documents"
os.chdir(new_directory)

print(os.getcwd())

filepath = "./data/catB_train.parquet" 

data = pd.read_parquet(filepath)
print(data.head())
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

C:\Users\parma\OneDrive\Documents
          clntnum race_desc ctrycode_desc clttype stat_flag min_occ_date  \
19550  91b546e924   Chinese     Singapore       P    ACTIVE   2017-10-31   
4600   896bae548c   Chinese     Singapore       P    ACTIVE   2007-05-23   
13337  f364439ae6    Others     Singapore       P    ACTIVE   2019-08-31   
15074  70f319cfe1   Chinese     Singapore       P    ACTIVE   2021-10-18   
19724  2647a81328   Chinese     Singapore       P    ACTIVE   2018-07-20   

       cltdob_fix cltsex_fix  flg_substandard  flg_is_borderline_standard  \
19550  1974-05-09     Female              0.0                         0.0   
4600   1979-11-11       Male              0.0                         0.0   
13337  1976-01-28       Male              0.0                         0.0   
15074  1976-03-19     Female              0.0                         0.0   
19724  1995-07-31     Female              0.0                         0.0   

       ...  recency_giclaim  giclaim_cnt_succe

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [17]:
###...code...###
columnNames = ["clntnum", "ctrycode_desc", "stat_flag", "min_occ_date", "cltdob_fix", "cltsex_fix", 
               "flg_substandard", "flg_is_borderline_standard", "flg_is_revised_term", "flg_has_health_claim", "flg_gi_claim", "flg_is_proposal", 
               "is_dependent_in_at_least_1_policy", "annual_income_est", "tot_inforce_pols", "tot_cancel_pols", "f_ever_declined_la",
               "f_purchase_lh"]
data = data.loc[:, columnNames]

# Adding cltage and clt_ten cols
# Drop rows with incomplete data for `cltdob_fix`
data['cltdob_fix'] = data['cltdob_fix'].replace("None", pd.NaT)
cltdob_rows = data[data['cltdob_fix'].isna()]
data = data.dropna(subset=['cltdob_fix'])

# Drop rows with incomplete data for `min_occ_date`
data['min_occ_date'] = data['min_occ_date'].replace("None", pd.NaT)
occDate_rows = data[data['min_occ_date'].isna()]
data = data.dropna(subset=['min_occ_date'])

# Convert cltdob_fix to datetime format and compute the client's age
currentDate = datetime.now()
data["cltdob_fix"] = data["cltdob_fix"].map(
    lambda x: datetime.strptime(x, "%Y-%m-%d"))
data["cltage"] = data["cltdob_fix"].map(
    lambda x: ((currentDate - x).days/365.25))

In [18]:
###Data Analysis

##Filter the data for clients who are no longer active in SingLife
df_inactive = data[data['stat_flag'] != 'ACTIVE']
selected_colns = ["flg_substandard", "flg_is_borderline_standard", "flg_is_revised_term", "is_dependent_in_at_least_1_policy", "annual_income_est", "tot_inforce_pols", "tot_cancel_pols", "f_ever_declined_la",
               "f_purchase_lh"]
df_inactive = df_inactive.loc[:, selected_colns]

##Checking total number of NAs for each column

inforce_na = df_inactive['tot_inforce_pols'].isna().sum()
cancelled_na = df_inactive['tot_cancel_pols'].isna().sum()
declined_na = df_inactive["f_ever_declined_la"].isna().sum()
purchased_na = df_inactive["f_purchase_lh"].isna().sum()

#print(inforce_na), print(cancelled_na), print(declined_na), print(purchased_na)

##Replace NA values with 0 in the selected columns
colns_to_fill = ["tot_inforce_pols", "tot_cancel_pols", "f_ever_declined_la", "f_purchase_lh"]
df_inactive[colns_to_fill] = df_inactive[colns_to_fill].fillna(0)

income_mapping = {
    'D.30K-60K': 45000,
    'E.BELOW30K': 30000,
    'A.ABOVE200K': 200000,
    'C.60K-100K': 80000,
    'B.100K-200K': 150000
}

df_inactive["annual_income_est"] = df_inactive["annual_income_est"].replace(income_mapping)

colns_1 = ["annual_income_est", "tot_cancel_pols"]
colns_2 = ["annual_income_est", 'f_ever_declined_la']
colns_3 = ["annual_income_est", "f_purchase_lh"]
df4 = df_inactive.loc[:,colns_1].dropna()
df5 = df_inactive.loc[:, colns_2].dropna()
df6 = df_inactive.loc[:,colns_3].dropna()

#checking income and correlation with cancellation history
print(df4.dtypes)
r1 = df4["annual_income_est"].corr(df4["tot_cancel_pols"])
print(r1)

#checking income and correlation with purchase inclination
r2 = df6["annual_income_est"].corr(df6["f_purchase_lh"])
print(r2)

#checking income and correlation with declined history
r3 = df5["annual_income_est"].corr(df5["f_ever_declined_la"])
print(r3)
#print(df4["annual_income_est"].unique())

annual_income_est    float64
tot_cancel_pols      float64
dtype: object
-0.02590562848189608
0.009417747919448957
0.05014613443800301


In [19]:
###Data modelling
unique1 = data["flg_is_proposal"].unique()
unique2 = data["f_purchase_lh"].unique()

na1 = data["flg_is_proposal"].isna().sum()
na2 = data["f_purchase_lh"].isna().sum()

colns = ["flg_is_proposal", "f_purchase_lh"]
data[colns]= data[colns].fillna(0) #those that have NA values become 0 instead

data['conversion'] = np.where((data["flg_is_proposal"] == 1) & (data["f_purchase_lh"] == 1), True, False)

#Convert income est from string to integers instead
income_mapping = {
    'D.30K-60K': 45000,
    'E.BELOW30K': 30000,
    'A.ABOVE200K': 200000,
    'C.60K-100K': 80000,
    'B.100K-200K': 150000
}

data["annual_income_est"] = data["annual_income_est"].replace(income_mapping)
cleaned_df = data.dropna(subset=['annual_income_est', 'cltage'])

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score



X = cleaned_df[["annual_income_est", "cltage"]]
Y = cleaned_df['conversion']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)



# Train the KNN model
k = 3
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred)

# Train the logistic regression model
logistic_reg_model = LogisticRegression()
logistic_reg_model.fit(X_train, y_train)
y_pred = logistic_reg_model.predict(X_test)
accuracy_logistic = accuracy_score(y_test, y_pred)
print(f'Accuracy of Logistic Regression model: {accuracy_logistic}')

Accuracy of Logistic Regression model: 0.9989001319841619


## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [20]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [21]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

[]


### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!